In [3]:
# !pip install --upgrade pip
# !pip install -q unsloth peft trl wandb evaluate jsonlines human-eval
# !pip install -q datasets transformers accelAerate einops tiktoken tqdm
# !pip uninstall xformers -y
# !pip install xformers
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import json
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer, DPOConfig, DPOTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM, PeftModel
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
import wandb
from pathlib import Path
import re
import torch
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
import os
from tqdm import tqdm

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Step 1: Load model and tokenizer with Unsloth (4-bit quantization)
model_id = "unsloth/Qwen2.5-Coder-3B-Instruct"
max_seq_length = 4096  # Adjust based on your needs (Qwen supports up to 32k, but for efficiency)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    dtype=None,  # Auto-detect (bfloat16 if supported)
    load_in_4bit=True,  # Enable 4-bit quantization
)
# Load tokenizer đúng của phiên bản INSTRUCT
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/Qwen2.5-Coder-3B-Instruct",
    trust_remote_code=True
)

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [5]:
# Load dataset JSONL
dataset = Dataset.from_json("/content/drive/MyDrive/QwenNoQuen/problems_train_cleaned.jsonl")


# 2️⃣ KHẮC PHỤC LỖI “UNTRAINED TOKENS”
# -----------------------------------------------------
# ❗ Với bản INSTRUCT → được phép dùng chat template
# ❗ KHÔNG được dùng chat template khi model là *base*
# -----------------------------------------------------
def format_chat(example):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful coding assistant. "
                "Complete the function based on the problem description and starter code."
            )
        },
        {
            "role": "user",
            "content": (
                f"Problem Description:\n{example['problem_description']}\n\n"
                f"Boilerplate code:\n{example['starter_code']}\n\n"
                f"Please complete the function."
            )
        },
        {
            "role": "assistant",
            "content": example["completion"]
        }
    ]

    # apply_chat_template: CHỈ DÙNG CHO BẢN INSTRUCT
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # Không thêm thừa <|im_start|>assistant
    )
    return {"text": formatted_text}


# Format toàn dataset
formatted_dataset = dataset.map(format_chat, num_proc=4)

# Kiểm tra 1 sample
print(formatted_dataset[0]["text"])

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/2641 [00:00<?, ? examples/s]

<|im_start|>system
You are a helpful coding assistant. Complete the function based on the problem description and starter code.<|im_end|>
<|im_start|>user
Problem Description:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
 
Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = [3,3], target = 6
Output: [0,1]

 
Constraints:

2 <= nums.length <= 104
-109 <= nums[i] <= 109
-109 <= target <= 109
Only one valid answer exists.

 
Follow-up: Can you come up with an algorithm that is less than O(n2) time complexity?

Boilerplate code:
def twoSum(self, nums: List[int], target: int) -> List[int]:
    

Please complet

In [6]:
# Step 3: Apply LoRA adapters via Unsloth
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank of LoRA adapters
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Efficient checkpointing
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.11.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [7]:
save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"
# Step 5: Set up training arguments
training_args = TrainingArguments(
    output_dir = save_dir,
    num_train_epochs=1.3,  # Few epochs to avoid overfitting on 2800 samples
    per_device_train_batch_size=2,  # Adjust based on GPU memory
    gradient_accumulation_steps=8,  # Effective batch size 16
    learning_rate=2e-5,
    warmup_ratio=0.05,   # ~5% của total steps
    lr_scheduler_type="cosine",  # Cosine decay tốt hơn linear
    weight_decay=0.01,
    max_grad_norm = 1.0,
    save_strategy="epoch",
    save_total_limit=2,
    fp16=False,
    bf16=True,
    report_to="wandb",
    logging_steps=5,  # Log mỗi 5 steps
    optim="adamw_8bit",
    seed = 42,
)

In [8]:
# Step 6: Initialize SFTTrainer for supervised fine-tuning with EarlyStoppingCallback
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset = formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/2641 [00:00<?, ? examples/s]

In [9]:
wandb.login()
wandb.init(project="demo-colab", name="demo1") #122e74ba5f2e5ab0922497dad7da9ee2f78fe1d4

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 23020356 (23020356-vnu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [10]:
# Step 7: Train the model
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,641 | Num Epochs = 2 | Total steps = 216
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,0.943400
10,0.929300
15,0.912200
20,0.861400
25,0.810600
30,0.813800
35,0.759600
40,0.689200
45,0.612700
50,0.555500


TrainOutput(global_step=216, training_loss=0.3780340520596063, metrics={'train_runtime': 1670.2677, 'train_samples_per_second': 2.056, 'train_steps_per_second': 0.129, 'total_flos': 3.770371426679194e+16, 'train_loss': 0.3780340520596063, 'epoch': 1.3028009084027252})

In [11]:
# Save LoRA checkpoint
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/tokenizer_config.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/special_tokens_map.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/chat_template.jinja',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/vocab.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/merges.txt',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/added_tokens.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora/tokenizer.json')

# Train DPO

In [14]:
# Đường dẫn LoRA SFT mà bạn đã lưu
sft_lora_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"

# Đường dẫn để lưu checkpoint DPO mới
dpo_output_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo"

# Nếu bạn vẫn nhớ tên base model ban đầu (trước khi SFT), để làm ref model:
base_model_name = "unsloth/Qwen2.5-Coder-3B-Instruct"  # ví dụ, sửa lại thành model bạn dùng thật

dpo_data_path = "/content/drive/MyDrive/QwenNoQuen/dpo_train.jsonl"
max_seq_length = 4096   # hoặc giá trị bạn đang dùng


In [15]:
# 1. Load base model + tokenizer
policy_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model_name,   # ví dụ: "unsloth/Qwen2.5-Coder-3B-Instruct"
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# 2. Gắn LoRA đã SFT vào base model
policy_model = PeftModel.from_pretrained(
    policy_model,
    sft_lora_dir,   # thư mục bạn đã save LoRA sau SFT
    is_trainable = True,  # vì ta muốn tiếp tục train DPO
)

==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
# ref_model = base model (chưa gắn LoRA/sau SFT)
ref_model, _ = FastLanguageModel.from_pretrained(
    model_name = base_model_name,   # cùng model như lúc đầu
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# Đóng băng ref_model
for param in ref_model.parameters():
    param.requires_grad = False


==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [17]:
# Đảm bảo có cả pad_token và eos_token
if tokenizer.eos_token is None:
    tokenizer.eos_token = "<|im_end|>"  # Qwen 2.5 standard

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [18]:
# ==================== 4. LOAD & FORMAT DPO DATA ====================
print("Loading DPO dataset...")
raw_dataset = load_dataset("json", data_files={"train": dpo_data_path})["train"]

print("Sample raw data:")
print(raw_dataset[0])

def format_dpo(example):
    messages_wo_assistant = [
        {
            "role": "system",
            "content": (
                "You are a helpful coding assistant. "
                "Complete the function based on the problem description and starter code."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Problem Description:\n{example['problem_description']}\n\n"
                f"Starter Code:\n{example['starter_code']}\n\n"
                "Please write the complete and correct solution."
            ),
        },
    ]

    # Tạo prompt theo chat template Qwen + thêm header assistant
    prompt = tokenizer.apply_chat_template(
        messages_wo_assistant,
        tokenize=False,
        add_generation_prompt=True,
    )

    def add_eos(text):
        text = text.rstrip()
        if tokenizer.eos_token and not text.endswith(tokenizer.eos_token):
            return text + tokenizer.eos_token
        return text

    chosen = add_eos(example["chosen"])
    rejected = add_eos(example["rejected"])

    return {
        "prompt": prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

print("Formatting DPO dataset...")
formatted_dataset = raw_dataset.map(format_dpo, remove_columns=raw_dataset.column_names)

# Kiểm tra sample
print("\n==== PROMPT SAMPLE ====")
print(formatted_dataset[28]["prompt"])
print("\n==== CHOSEN SAMPLE ====")
print(formatted_dataset[28]["chosen"])
print("\n==== REJECTED SAMPLE ====")
print(formatted_dataset[28]["rejected"])

# Train/eval split
splits = formatted_dataset.train_test_split(test_size=0.08, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]

print(f"\nTrain samples: {len(train_dataset)}, Eval samples: {len(eval_dataset)}")


Loading DPO dataset...
Sample raw data:
{'question_id': 3243, 'problem_description': 'You are given an integer n and a 2D integer array queries.\nThere are n cities numbered from 0 to n - 1. Initially, there is a unidirectional road from city i to city i + 1 for all 0 <= i < n - 1.\nqueries[i] = [ui, vi] represents the addition of a new unidirectional road from city ui to city vi. After each query, you need to find the length of the shortest path from city 0 to city n - 1.\nReturn an array answer where for each i in the range [0, queries.length - 1], answer[i] is the length of the shortest path from city 0 to city n - 1 after processing the first i + 1 queries.\n\xa0\nExample 1:\n\nInput: n = 5, queries = [[2,4],[0,2],[0,4]]\nOutput: [3,2,1]\nExplanation: \n\nAfter the addition of the road from 2 to 4, the length of the shortest path from 0 to 4 is 3.\n\nAfter the addition of the road from 0 to 2, the length of the shortest path from 0 to 4 is 2.\n\nAfter the addition of the road from 

In [19]:
wandb.login()
wandb.init(project="demo-DPO", name="demoSS")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


eval/logits/chosen,▁█
eval/logits/rejected,▁█
eval/logps/chosen,▁█
eval/logps/rejected,█▁
eval/loss,█▁
eval/rewards/accuracies,▁█
eval/rewards/chosen,▁█
eval/rewards/margins,▁█
eval/rewards/rejected,█▁
eval/runtime,▁█
+15,...


In [20]:
dpo_config = DPOConfig(
    output_dir = dpo_output_dir,
    per_device_train_batch_size = 2,   # nếu GPU yếu, cứ để nhỏ
    gradient_accumulation_steps = 8,   # tăng tích lũy để có effective batch lớn hơn
    num_train_epochs = 5,              # 2–3 epoch là hợp lý với 500 cặp
    learning_rate = 5e-6,              # nhỏ hơn SFT một chút
    warmup_ratio=0.05,                      # ✅ Thêm warmup
    weight_decay=0.01,                    # ✅ Regularization
    logging_steps = 1,
    save_strategy = "epoch",
    bf16 = torch.cuda.is_available(),  # nếu GPU hỗ trợ bf16 thì dùng
    gradient_checkpointing = True,
    eval_strategy = "steps",
    eval_steps = 10,
    optim = "adamw_8bit",        # phù hợp với 4bit
    report_to="wandb",
    seed = 42,
)
dpo_config.padding_value = tokenizer.pad_token_id

In [21]:
dpo_trainer = DPOTrainer(
    model = policy_model,
    ref_model = ref_model,
    beta = 0.2,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    args = dpo_config,
    max_length = 4096,           # Tổng chiều dài
    max_prompt_length = 2048,    # ✅ Giới hạn prompt để còn chỗ cho response
    max_target_length = 2048,    # ✅ Giới hạn response
)

In [22]:
dpo_trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 419 | Num Epochs = 5 | Total steps = 135
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.590100,0.683143,1.038783,0.856181,0.700000,0.182602,-170.839447,-45.576580,-3.365418,-3.547480,0,0,0
20,0.552800,0.590882,1.093068,0.716637,0.750000,0.376431,-170.296585,-46.972023,-3.348031,-3.510906,No Log,No Log,No Log
30,0.618000,0.525424,1.152172,0.607832,0.825000,0.544341,-169.705536,-48.060081,-3.335611,-3.485036,No Log,No Log,No Log
40,0.502600,0.473745,1.175962,0.485667,0.850000,0.690295,-169.467636,-49.281719,-3.325441,-3.466071,No Log,No Log,No Log
50,0.515900,0.436020,1.210265,0.402618,0.875000,0.807647,-169.124619,-50.112221,-3.319553,-3.452800,No Log,No Log,No Log
60,0.469700,0.404009,1.271813,0.359326,0.900000,0.912488,-168.509125,-50.545135,-3.312986,-3.439584,No Log,No Log,No Log
70,0.392200,0.379688,1.303122,0.300679,0.925000,1.002443,-168.196045,-51.131603,-3.305811,-3.428743,No Log,No Log,No Log
80,0.295600,0.358886,1.349032,0.266583,0.950000,1.082449,-167.736954,-51.472572,-3.303253,-3.422233,No Log,No Log,No Log
90,0.285100,0.344609,1.372713,0.229846,0.975000,1.142866,-167.500153,-51.839928,-3.302706,-3.420306,No Log,No Log,No Log
100,0.336300,0.334097,1.395929,0.199811,0.975000,1.196118,-167.267990,-52.140289,-3.300764,-3.415954,No Log,No Log,No Log


TrainOutput(global_step=135, training_loss=0.43805421436274494, metrics={'train_runtime': 3434.6373, 'train_samples_per_second': 0.61, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 0.43805421436274494, 'epoch': 5.0})

In [23]:
# Nếu policy_model là PEFT model (có LoRA):
policy_model.save_pretrained(dpo_output_dir)
tokenizer.save_pretrained(dpo_output_dir)


('/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/tokenizer_config.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/special_tokens_map.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/chat_template.jinja',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/vocab.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/merges.txt',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/added_tokens.json',
 '/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo/tokenizer.json')

In [ ]:
# ==================== 1. CONFIG ====================
base_model_name = "unsloth/Qwen2.5-Coder-3B-Instruct"
dpo_output_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo"

PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/demo"       # folder chứa problem*.md
OUTPUT_DIR   = "/content/drive/MyDrive/QwenNoQuen/solution"   # folder lưu solution*.py
os.makedirs(OUTPUT_DIR, exist_ok=True)

max_seq_length = 4096
device = "cuda" if torch.cuda.is_available() else "cpu"

# ==================== 2. LOAD MODEL + LoRA DPO ====================
print("Loading model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name      = base_model_name,
    max_seq_length  = max_seq_length,
    dtype           = None,
    load_in_4bit    = True,
)

model = PeftModel.from_pretrained(
    model,
    dpo_output_dir,
    is_trainable = False,   # inference mode
)

FastLanguageModel.for_inference(model)

# Đảm bảo eos / pad token chuẩn
if tokenizer.eos_token is None:
    tokenizer.eos_token = "<|im_end|>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# ==================== 3. HELPER FUNCTIONS ====================

def parse_problem_markdown(md_text: str):
    """
    Tách problem_description và starter_code từ file .md.
    Giả định:
    - Mô tả đề bài nằm trước code block đầu tiên
    - Starter code nằm trong code block đầu tiên (```python ... ``` hoặc ``` ... ```).
    """
    # Tìm code block ```python ... ``` hoặc ``` ... ```
    code_block_pattern = re.compile(
        r"```(?:python)?\s*(.*?)```",
        re.DOTALL | re.IGNORECASE
    )
    match = code_block_pattern.search(md_text)

    if match:
        starter_code = match.group(1).strip()
        problem_description = md_text[:match.start()].strip()
    else:
        # Không tìm thấy code block -> coi toàn bộ là problem_description, starter_code rỗng
        problem_description = md_text.strip()
        starter_code = ""

    return problem_description, starter_code


def build_inference_prompt(problem_description: str, starter_code: str) -> str:
    """
    Xây prompt y HỆT format đã dùng khi train DPO.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful coding assistant. "
                "Complete the function based on the problem description and starter code."
            ),
        },
        {
            "role": "user",
            "content": (
                f"Problem Description:\n{problem_description}\n\n"
                f"Starter Code:\n{starter_code}\n\n"
                "Please write the complete and correct solution."
            ),
        },
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,   # thêm <|im_start|>assistant
    )
    return prompt


def extract_assistant_output(full: str) -> str:
    """
    Lấy phần assistant DUY NHẤT, bỏ system + user prompt.
    """
    if "<|im_start|>assistant" in full:
        return full.split("<|im_start|>assistant", 1)[-1].strip()
    return full.strip()


def clean_code(text: str) -> str:
    """
    Xóa markdown code fence, token đặc biệt, khoảng trắng thừa.
    """
    # Xóa ```python ở đầu
    text = re.sub(r"^```python\s*", "", text, flags=re.MULTILINE | re.IGNORECASE)
    # Xóa ``` ở đầu/cuối
    text = re.sub(r"^```\s*", "", text, flags=re.MULTILINE)
    text = re.sub(r"```\s*$", "", text, flags=re.MULTILINE)
    # Xóa token đặc biệt (nếu lỡ dính)
    text = text.replace("<|im_end|>", "").replace("<|im_start|>", "")
    # Trim
    text = text.strip()
    return text


def get_problem_index_from_filename(path: Path) -> str:
    """
    Từ 'problem39.md' -> '39'.
    Tránh dùng strip('problem') vì sẽ remove ký tự theo set.
    """
    name = path.stem  # 'problem39'
    prefix = "problem"
    if name.lower().startswith(prefix):
        return name[len(prefix):]
    return name  # fallback


# ==================== 4. DUYỆT QUA TẤT CẢ PROBLEM*.MD ====================
problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

for problem_file in problem_files:
    with open(problem_file, "r", encoding="utf-8") as f:
        md_content = f.read()

    # 1. Parse đề + starter code từ markdown
    problem_description, starter_code = parse_problem_markdown(md_content)

    # 2. Build prompt giống hệt lúc train DPO
    prompt = build_inference_prompt(problem_description, starter_code)

    # 3. Tokenize & generate
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens = 512,
            do_sample      = False,                  # greedy để ổn định, thử xong rồi hãy bật sampling
            eos_token_id   = tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)
    assistant_only = extract_assistant_output(decoded)
    code = clean_code(assistant_only)

    # 4. Lưu thành file .py
    idx = get_problem_index_from_filename(problem_file)
    output_file = os.path.join(OUTPUT_DIR, f"solution{idx}.py")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(code)

    print(f"✅ Đã giải và lưu bài: {problem_file.name} -> solution{idx}.py")

print(f"\n🎉 Hoàn thành! Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")


Loading model...
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📁 Tìm thấy 50 bài toán

✅ Đã giải và lưu bài: problem10.md -> solution10.py
✅ Đã giải và lưu bài: problem41.md -> solution41.py
✅ Đã giải và lưu bài: problem15.md -> solution15.py
✅ Đã giải và lưu bài: problem16.md -> solution16.py
✅ Đã giải và lưu bài: problem42.md -> solution42.py
✅ Đã giải và lưu bài: problem48.md -> solution48.py
✅ Đã giải và lưu bài: problem1.md -> solution1.py
✅ Đã giải và lưu bài: problem11.md -> solution11.py
✅ Đã giải và lưu bài: problem28.md -> solution28.py
✅ Đã giải và lưu bài: problem49.md -

In [18]:
# # ==================== 1. LOAD MODEL ====================
# base_model_name = "unsloth/Qwen2.5-Coder-3B-Instruct"
# dpo_output_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora_dpo"
# max_seq_length = 4096

# print("Loading model...")
# # Load base model
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name=base_model_name,
#     max_seq_length=max_seq_length,
#     dtype=None,
#     load_in_4bit=True,
# )

# # Load LoRA đã DPO
# model = PeftModel.from_pretrained(
#     model,
#     dpo_output_dir,
#     is_trainable=False,  # ✅ Inference mode
# )

# # Bật FastLanguageModel để tăng tốc inference
# FastLanguageModel.for_inference(model)

# PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/demo"
# OUTPUT_DIR = "/content/drive/MyDrive/QwenNoQuen/solution"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
# print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

# def clean_code(text):
#     """
#     Xóa các phần đánh dấu markdown code block và các thẻ đặc biệt
#     """
#     # Xóa ```python ở đầu
#     text = re.sub(r'^```python\s*', '', text, flags=re.MULTILINE)

#     # Xóa ```<|im_end|> ở cuối
#     text = re.sub(r'```<\|im_end\|>\s*$', '', text, flags=re.MULTILINE)

#     # Xóa ``` đơn thuần (nếu có)
#     text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
#     text = re.sub(r'```\s*$', '', text, flags=re.MULTILINE)

#     # Xóa <|im_end|> đơn thuần (nếu có)
#     text = re.sub(r'<\|im_end\|>', '', text)

#     # Xóa khoảng trắng thừa ở đầu và cuối
#     text = text.strip()

#     return text

# def extract_assistant_output(full):
#     """
#     Lấy phần assistant DUY NHẤT, bỏ hệ thống + user prompt.
#     """
#     if "<|im_start|>assistant" in full:
#         return full.split("<|im_start|>assistant")[-1].strip()
#     # fallback if no special tokens
#     return full.strip()

# for problem_file in problem_files:
#     # đọc đề bài
#     with open(problem_file, 'r', encoding='utf-8') as f:
#         problem_content = f.read()

#     # 1. Lấy tên file (problemX.md)
#     filename = os.path.basename(problem_file)

#     # 2. Loại bỏ phần mở rộng (.md)
#     filename_without_ext = os.path.splitext(filename)[0] # Kết quả: problemx

#     # 3. Loại bỏ tiền tố "problem"
#     result_index = filename_without_ext.strip('problem') # Kết quả: 39

#     # ====== ĐÚNG CHUẨN: TẠO CHAT TEMPLATE ======
#     messages = [
#         {"role": "system", "content": "You are a helpful coding assistant."},
#         {"role": "user", "content": problem_content +
#          "\n\nUse knowledge of data structures and algorithms. Think step by step to solve the code. Please provide ONLY Python function implementation, no explanation needed."}
#     ]

#     prompt = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Run model
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512,
#         #temperature=0.1,
#         #top_p=0.9,
#         do_sample=False,
#         eos_token_id=tokenizer.eos_token_id
#     )

#     decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

#     # ====== TÁCH RIÊNG PHẦN HÀM PYTHON ======
#     assistant_only = extract_assistant_output(decoded)

#     # ====== THAY ĐỔI: LƯU THÀNH FILE .py ======
#     output_file = os.path.join(OUTPUT_DIR, f"solution{result_index}.py")
#     with open(output_file, 'w', encoding='utf-8') as f:
#         f.write(clean_code(assistant_only))
#     print(f"Đã giải bài thứ: {result_index}")
# print(f"\n✅ Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")

Loading model...
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📁 Tìm thấy 50 bài toán

Đã giải bài thứ: 10
Đã giải bài thứ: 41
Đã giải bài thứ: 15
Đã giải bài thứ: 16
Đã giải bài thứ: 42
Đã giải bài thứ: 48
Đã giải bài thứ: 1
Đã giải bài thứ: 11
Đã giải bài thứ: 28
Đã giải bài thứ: 49
Đã giải bài thứ: 43
Đã giải bài thứ: 13
Đã giải bài thứ: 32
Đã giải bài thứ: 45
Đã giải bài thứ: 47
Đã giải bài thứ: 37
Đã giải bài thứ: 20
Đã giải bài thứ: 24
Đã giải bài thứ: 19
Đã giải bài thứ: 3
Đã giải bài thứ: 7
Đã giải bài thứ: 36
Đã giải bài thứ: 46
Đã giải bài thứ: 22
Đã giải bài thứ: 31
Đã gi

KeyboardInterrupt: 

In [13]:
# import re
# import os
# from pathlib import Path
# from tqdm import tqdm

# # Load LoRA model
# save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"
# model, tokenizer = FastLanguageModel.from_pretrained(
#     save_dir,
#     max_seq_length=max_seq_length,
#     load_in_4bit=True,
# )

# PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/test"
# OUTPUT_DIR = "/content/drive/MyDrive/QwenNoQuen/solution"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
# print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

# def clean_code(text):
#     """
#     Xóa các phần đánh dấu markdown code block và các thẻ đặc biệt
#     """
#     # Xóa ```python ở đầu
#     text = re.sub(r'^```python\s*', '', text, flags=re.MULTILINE)

#     # Xóa ```<|im_end|> ở cuối
#     text = re.sub(r'```<\|im_end\|>\s*$', '', text, flags=re.MULTILINE)

#     # Xóa ``` đơn thuần (nếu có)
#     text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
#     text = re.sub(r'```\s*$', '', text, flags=re.MULTILINE)

#     # Xóa <|im_end|> đơn thuần (nếu có)
#     text = re.sub(r'<\|im_end\|>', '', text)

#     # Xóa khoảng trắng thừa ở đầu và cuối
#     text = text.strip()

#     return text

# def extract_assistant_output(full):
#     """
#     Lấy phần assistant DUY NHẤT, bỏ hệ thống + user prompt.
#     """
#     if "<|im_start|>assistant" in full:
#         return full.split("<|im_start|>assistant")[-1].strip()
#     # fallback if no special tokens
#     return full.strip()

# for problem_file in problem_files:
#     # đọc đề bài
#     with open(problem_file, 'r', encoding='utf-8') as f:
#         problem_content = f.read()

#     # 1. Lấy tên file (problemX.md)
#     filename = os.path.basename(problem_file)

#     # 2. Loại bỏ phần mở rộng (.md)
#     filename_without_ext = os.path.splitext(filename)[0] # Kết quả: problemx

#     # 3. Loại bỏ tiền tố "problem"
#     result_index = filename_without_ext.strip('problem') # Kết quả: 39

#     # ====== ĐÚNG CHUẨN: TẠO CHAT TEMPLATE ======
#     messages = [
#         {"role": "system", "content": "You are a helpful coding assistant."},
#         {"role": "user", "content": problem_content +
#          "\n\nUse knowledge of data structures and algorithms. Think step by step to solve the code. Please provide ONLY Python function implementation, no explanation needed."}
#     ]

#     prompt = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Run model
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512,
#         temperature=0.7,
#         top_p=0.9,
#         do_sample=True,
#         eos_token_id=tokenizer.eos_token_id
#     )

#     decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

#     # ====== TÁCH RIÊNG PHẦN HÀM PYTHON ======
#     assistant_only = extract_assistant_output(decoded)

#     # ====== THAY ĐỔI: LƯU THÀNH FILE .py ======
#     output_file = os.path.join(OUTPUT_DIR, f"solution{result_index}.py")
#     with open(output_file, 'w', encoding='utf-8') as f:
#         f.write(clean_code(assistant_only))
#     print(f"Đã giải bài thứ: {result_index}")
# print(f"\n✅ Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")

In [14]:
# import re
# import os
# from pathlib import Path
# from tqdm import tqdm

# # Load LoRA model
# save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"
# model, tokenizer = FastLanguageModel.from_pretrained(
#     save_dir,
#     max_seq_length=max_seq_length,
#     load_in_4bit=True,
# )

# PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/problems_test"
# OUTPUT_DIR = "/content/drive/MyDrive/QwenNoQuen/solution1"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
# print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

# def clean_code(text):
#     """
#     Xóa các phần đánh dấu markdown code block và các thẻ đặc biệt
#     """
#     # Xóa ```python ở đầu
#     text = re.sub(r'^```python\s*', '', text, flags=re.MULTILINE)

#     # Xóa ```<|im_end|> ở cuối
#     text = re.sub(r'```<\|im_end\|>\s*$', '', text, flags=re.MULTILINE)

#     # Xóa ``` đơn thuần (nếu có)
#     text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
#     text = re.sub(r'```\s*$', '', text, flags=re.MULTILINE)

#     # Xóa <|im_end|> đơn thuần (nếu có)
#     text = re.sub(r'<\|im_end\|>', '', text)

#     # Xóa khoảng trắng thừa ở đầu và cuối
#     text = text.strip()

#     return text

# def extract_assistant_output(full):
#     """
#     Lấy phần assistant DUY NHẤT, bỏ hệ thống + user prompt.
#     """
#     if "<|im_start|>assistant" in full:
#         return full.split("<|im_start|>assistant")[-1].strip()
#     # fallback if no special tokens
#     return full.strip()

# for problem_file in problem_files:
#     # đọc đề bài
#     with open(problem_file, 'r', encoding='utf-8') as f:
#         problem_content = f.read()

#     # 1. Lấy tên file (problemX.md)
#     filename = os.path.basename(problem_file)

#     # 2. Loại bỏ phần mở rộng (.md)
#     filename_without_ext = os.path.splitext(filename)[0] # Kết quả: problemx

#     # 3. Loại bỏ tiền tố "problem"
#     result_index = filename_without_ext.strip('problem') # Kết quả: 39

#     # ====== ĐÚNG CHUẨN: TẠO CHAT TEMPLATE ======
#     messages = [
#         {"role": "system", "content": "You are a helpful coding assistant."},
#         {"role": "user", "content": problem_content +
#          "\n\nUse knowledge of data structures and algorithms. Think step by step to solve the code. Please provide ONLY Python function implementation, no explanation needed."}
#     ]

#     prompt = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Run model
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512,
#         temperature=0.4,
#         top_p=0.9,
#         do_sample=True,
#         eos_token_id=tokenizer.eos_token_id
#     )

#     decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

#     # ====== TÁCH RIÊNG PHẦN HÀM PYTHON ======
#     assistant_only = extract_assistant_output(decoded)
#     cleaned_solution = clean_code(assistant_only)

#     # ====== THAY ĐỔI: LƯU THÀNH FILE .py ======
#     output_file = os.path.join(OUTPUT_DIR, f"solution{result_index}.jsonl")
#     with open(output_file, "w", encoding="utf-8") as f:
#         json_line = {
#             "problem_id": result_index,
#             "solution": cleaned_solution
#         }
#         f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

#     print(f"Đã giải bài thứ: {result_index}")
# print(f"\n✅ Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")

In [15]:
# import re
# import os
# from pathlib import Path
# from tqdm import tqdm

# # Load LoRA model
# save_dir = "/content/drive/MyDrive/QwenNoQuen/fine_tuned_qwen_lora"
# model, tokenizer = FastLanguageModel.from_pretrained(
#     save_dir,
#     max_seq_length=max_seq_length,
#     load_in_4bit=True,
# )

# PROBLEMS_DIR = "/content/drive/MyDrive/QwenNoQuen/problems_test"
# OUTPUT_DIR = "/content/drive/MyDrive/QwenNoQuen/solution2"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# problem_files = list(Path(PROBLEMS_DIR).glob("problem*.md"))
# print(f"📁 Tìm thấy {len(problem_files)} bài toán\n")

# def clean_code(text):
#     """
#     Xóa các phần đánh dấu markdown code block và các thẻ đặc biệt
#     """
#     # Xóa ```python ở đầu
#     text = re.sub(r'^```python\s*', '', text, flags=re.MULTILINE)

#     # Xóa ```<|im_end|> ở cuối
#     text = re.sub(r'```<\|im_end\|>\s*$', '', text, flags=re.MULTILINE)

#     # Xóa ``` đơn thuần (nếu có)
#     text = re.sub(r'^```\s*', '', text, flags=re.MULTILINE)
#     text = re.sub(r'```\s*$', '', text, flags=re.MULTILINE)

#     # Xóa <|im_end|> đơn thuần (nếu có)
#     text = re.sub(r'<\|im_end\|>', '', text)

#     # Xóa khoảng trắng thừa ở đầu và cuối
#     text = text.strip()

#     return text

# def extract_assistant_output(full):
#     """
#     Lấy phần assistant DUY NHẤT, bỏ hệ thống + user prompt.
#     """
#     if "<|im_start|>assistant" in full:
#         return full.split("<|im_start|>assistant")[-1].strip()
#     # fallback if no special tokens
#     return full.strip()

# for problem_file in problem_files:
#     # đọc đề bài
#     with open(problem_file, 'r', encoding='utf-8') as f:
#         problem_content = f.read()

#     # 1. Lấy tên file (problemX.md)
#     filename = os.path.basename(problem_file)

#     # 2. Loại bỏ phần mở rộng (.md)
#     filename_without_ext = os.path.splitext(filename)[0] # Kết quả: problemx

#     # 3. Loại bỏ tiền tố "problem"
#     result_index = filename_without_ext.strip('problem') # Kết quả: 39

#     # ====== ĐÚNG CHUẨN: TẠO CHAT TEMPLATE ======
#     messages = [
#         {"role": "system", "content": "You are a helpful coding assistant."},
#         {"role": "user", "content": problem_content +
#          "\n\nUse knowledge of data structures and algorithms. Think step by step to solve the code. Please provide ONLY Python function implementation, no explanation needed."}
#     ]

#     prompt = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )

#     # Run model
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=1024,
#         temperature=0.2,
#         top_p=0.9,
#         do_sample=True,
#         eos_token_id=tokenizer.eos_token_id
#     )

#     decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

#     # ====== TÁCH RIÊNG PHẦN HÀM PYTHON ======
#     assistant_only = extract_assistant_output(decoded)
#     cleaned_solution = clean_code(assistant_only)

#     # ====== THAY ĐỔI: LƯU THÀNH FILE .py ======
#     output_file = os.path.join(OUTPUT_DIR, f"solution{result_index}.jsonl")
#     with open(output_file, "w", encoding="utf-8") as f:
#         json_line = {
#             "problem_id": result_index,
#             "solution": cleaned_solution
#         }
#         f.write(json.dumps(json_line, ensure_ascii=False) + "\n")

#     print(f"Đã giải bài thứ: {result_index}")
# print(f"\n✅ Đã lưu {len(problem_files)} solutions tại: {OUTPUT_DIR}")